In [16]:
import google.generativeai as genai
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

- https://ai.google.dev/gemini-api/docs/quickstart?lang=python
- https://github.com/anthropics/anthropic-cookbook/blob/main/misc/how_to_make_sql_queries.ipynb
- https://platform.openai.com/docs/guides/function-calling
- https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

In [2]:
load_dotenv()

GEMINI_KEY = os.getenv('GEMINI_API_KEY')

In [3]:
models = {
    "flash_2.0": "gemini-2.0-flash-exp",
}

In [4]:
#genai.configure(api_key=GEMINI_KEY)
#model = genai.GenerativeModel(models["flash_2.0"])
#response = model.generate_content("Explain how AI works")
#print(response.text)

In [5]:
client = OpenAI(
    api_key=GEMINI_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [6]:
response = client.chat.completions.create(
    model=models["flash_2.0"],
    n=1,
    messages=[
        {
            "role": "system", 
            "content": "You are a helpful beginner math bot. You can perform addition and subtraction using two input values. Do not perform any other tasks."
        },
        {
            "role": "user", 
            "content": "What is 5 plus 3?"},
    ]
)

print(response.choices[0].message)

ChatCompletionMessage(content='5 + 3 = 8\n', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


## Tools (functions)

In [7]:
def addition(a: int, b: int) -> int:
    return a+b

def subtraction(a: int, b: int) -> int:
    return a-b


math_controls = [addition, subtraction]
instruction = "You are a helpful beginner math bot. You can perform addition and subtraction using two input values. Do not perform any other tasks."

In [56]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "addition",
            "description": "Add two numbers",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "number",
                        "description": "First number"
                    },
                    "b": {
                        "type": "number",
                        "description": "Second number"
                    }
                },
                "required": ["a", "b"],
                "additionalProperties": False,
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "subtraction",
            "description": "Subtract two numbers",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "number",
                        "description": "First number"
                    },
                    "b": {
                        "type": "number",
                        "description": "Second number"
                    }
                },
                "required": ["a", "b"],
                "additionalProperties": False
            }
        }
    }
]

instruction = "You are a helpful beginner math bot. You can perform addition and subtraction using two input values. Do not perform any other tasks."
prompt = "What is 6 plus 4?"
# prompt = "How old is the king?"

messages = [
    {
        "role": "user",
        "content": prompt
    },
    {
        "role": "system",
        "content": instruction
    }
]

response = client.chat.completions.create(
    model=models["flash_2.0"],
    messages=messages,
    tools=tools,
    # tool_choice='auto'
)

print(response)


ChatCompletion(id=None, choices=[Choice(finish_reason=None, index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, toolCalls=[{'function': {'arguments': '{"a":6,"b":4}', 'name': 'addition'}, 'id': '0', 'type': 'function'}]), finishReason='stop')], created=1736173617, model='gemini-2.0-flash-exp', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=None, prompt_tokens=None, total_tokens=None, completion_tokens_details=None, prompt_tokens_details=None, completionTokens=3, promptTokens=128, totalTokens=131))


In [57]:
print(f"completion tokens: {response.usage.completionTokens}")
print(f"prompt tokens: {response.usage.promptTokens}")
print(f"total tokens: {response.usage.totalTokens}")

completion tokens: 3
prompt tokens: 128
total tokens: 131


In [58]:
type(response.choices[0])

openai.types.chat.chat_completion.Choice

In [59]:
#response.choices[0].message.toolCalls

In [60]:
#response.choices[0].message.toolCalls[0]['function']['name']

In [61]:
#json.dumps(response.choices[0].message.toolCalls[0])

In [62]:
def check_if_toolcall(response):
    if hasattr(response.choices[0].message, 'toolCalls'):
        return True
    False

In [63]:
def call_function_from_output(response):
    # extract functino name and arguments
    function_call = response.choices[0].message.toolCalls[0]['function']
    function_name = function_call['name']
    args = eval(function_call['arguments'])

    function_lookup = {
        "addition": addition,
        "subtraction": subtraction
    }

    # call the appropriate function
    if function_name in function_lookup:
        result = function_lookup[function_name](**args)
        return result
    else:
        raise ValueError(f"unknown function '{function_name}'")

In [ ]:
def create_function_call_result_message():
    result_message = {
        "role": "tool",
        "content": json.dumps({
            "number_1": num1,
            "number_2": num2,
            "output": result
        }),
        
    }

In [65]:
if check_if_toolcall(response):
    pass
else:
    print("No function was called")

10
